In [1]:
cd /content/drive/MyDrive/OpenNMT-kpg-release-master/models

/content/drive/MyDrive/OpenNMT-kpg-release-master/models


In [2]:
! pip install datasets transformers rouge-score nltk

     |████████████████████████████████| 306 kB 5.2 MB/s 
     |████████████████████████████████| 3.4 MB 47.3 MB/s 
     |████████████████████████████████| 61 kB 471 kB/s 
     |████████████████████████████████| 243 kB 51.9 MB/s 
     |████████████████████████████████| 1.1 MB 41.1 MB/s 
     |████████████████████████████████| 132 kB 52.8 MB/s 
     |████████████████████████████████| 596 kB 46.0 MB/s 
     |████████████████████████████████| 3.3 MB 44.3 MB/s 
     |████████████████████████████████| 895 kB 47.0 MB/s 
     |████████████████████████████████| 192 kB 47.2 MB/s 
     |████████████████████████████████| 160 kB 51.3 MB/s 
     |████████████████████████████████| 271 kB 46.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!nvidia-smi

Mon Jan 10 16:39:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from itertools import chain
from nltk.corpus import wordnet

text = "exact penalty functions"
splited_text = (text.split(" "))
for splited_item in splited_text:
  synonyms = wordnet.synsets(splited_item)
  print (set(chain.from_iterable([word.lemma_names() for word in synonyms])))

{'precise', 'accurate', 'take', 'exact', 'demand', 'claim'}
{'penalty', 'penalisation', 'penalization', 'punishment'}
{'mathematical_function', 'work', 'use', 'occasion', 'social_function', 'function', 'subroutine', 'social_occasion', 'go', 'procedure', 'single-valued_function', 'map', 'operate', 'affair', 'part', 'mapping', 'purpose', 'role', 'run', 'routine', 'serve', 'officiate', 'office', 'subprogram'}


In [4]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
import json
import torch
import ast

device = 'cuda'#'cpu

model = BartForConditionalGeneration.from_pretrained('KPBL-1028').to(device)#BART-large-Finetuned
tokenizer = BartTokenizer.from_pretrained('KPBL-1028')
#tokenizer = BartTokenizer.from_pretrained('added')


In [5]:
ss = tokenizer("Keyphrases are condensed summaries of text information, representing a document's theme and core idea. Keyphrase extraction approaches based on statistics and deep learning have received widespread attention. However, the extractive approaches have obvious limitations like keyphrases must occur in the document. However, a considerable amount of human-annotated keyphrases are absent keyphrases such that they are not occurring in the source document explicitly, which cannot be extracted by the extractive methods. Keyphrase generation is based on a sequence-to-sequence deep learning model, generating present and absent keyphrases from the semantic representation of the document. However, recall and precision of generated abstract keyphrases still need to be improved. We propose a two-step keyphrase generation model to solve these problems, which consists of a finetuned BART-based keyphrase generator, coupled with a BERT cross encoder which ranks generated keyphrases. Our model can generate absent keyphrases and shows superior results over the strong baselines in four widely used datasets.")['input_ids']

In [6]:
print (len(ss))

219


In [ ]:
tinputs = tokenizer(["nlp"], return_tensors='pt', truncation=True).to(device)#, padding=True
print (tinputs)

# Generate Summary
tsummary_ids = model.generate(tinputs['input_ids'],max_length = 256,min_length = 64,num_beams = 7).to(device)  #length_penalty = 3.0  top_k = 5
tbart_pred = str([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in tsummary_ids])#[2:-2]
print (tbart_pred)

{'input_ids': tensor([[    0, 35760,   642,     2]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1]], device='cuda:0')}
['["information storage and retrieval", "natural language processing", "artificial intelligence", "computer vision", "pattern recognition", "machine learning", "neural networks", "support vector machines", "genetic algorithms", "probabilistic models", "statistical models", "<digit> <digit>", "data mining"]']


In [ ]:
tokenizer.save_pretrained("added")

('added/tokenizer_config.json',
 'added/special_tokens_map.json',
 'added/vocab.json',
 'added/merges.txt',
 'added/added_tokens.json')

In [ ]:
tokenizer.save_vocabulary("added")

Vocabulary path (added) should be a directory


In [ ]:
tokenizer.vocab_size

50265

In [ ]:
print(len(tokenizer))

50265


In [ ]:
tokenizer.get_added_vocab()

{}

In [8]:
def bart_pred(src):
  ARTICLE_TO_SUMMARIZE = src
  inputs = tokenizer([ARTICLE_TO_SUMMARIZE], return_tensors='pt', truncation=True).to(device)#, padding=True


  # Generate Summary
  summary_ids = model.generate(inputs['input_ids'],max_length = 256,min_length = 64,num_beams = 7).to(device)  #length_penalty = 3.0  top_k = 5
  bart_pred = str([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in summary_ids])#[2:-2]
  return (bart_pred)

In [ ]:
cd /content/drive/MyDrive/OpenNMT-kpg-release-master/data/keyphrase/meng17/

/content/drive/MyDrive/OpenNMT-kpg-release-master/data/keyphrase/meng17


In [ ]:
#Original!
def get_pred_BART(dataset_name):
    data = []
    with open(dataset_name+ '/' +dataset_name+ "_test.src", 'r+') as f:
        for line in f:
            data.append(json.loads(line))
        global data_len
        data_len = len(data)
        for i in range(len(data)):
          dic = data[i]
          if i % 50 == 0:          
            print(dataset_name + " ：" + str(i) + " / " + str(len(data)))

          src_value = dic["src"]
          bart_tgt = bart_pred(src_value)
          dic["pred"] = bart_tgt.replace ('\\','')
          del dic['title']
          del dic['src']
          del dic['abstract']
          # print(dic)
          with open(dataset_name+ '/' +dataset_name+ "_BL1028.pred",'a+') as t:
              json.dump(dic,t)
              t.write('\n')
        f.close()
        t.close()
      

In [ ]:
#domain corpus xxx
def get_pred_BART_domain(dataset_name):
    data = []
    generated = []
    with open(dataset_name+ '/' +dataset_name+ "_test.src", 'r+') as f:
        for line in f:
            data.append(json.loads(line))
        global data_len
        data_len = len(data)
        for i in range(len(data)):
          dic = data[i]
          if i % 50 == 0:          
            print(dataset_name + " ：" + str(i) + " / " + str(len(data)))
            

          src_value = dic["src"]# + str(generated)
          bart_tgt = bart_pred(src_value)[2:-2]
          

          try:
              generated.append(ast.literal_eval(bart_tgt))
          except:
              print("list error")
          
          
          # generated += domain_tgt[0:2]
          # generated = set(generated)
          # generated = list(generated)

          dic["pred"] = bart_tgt.replace ('\\','')
          del dic['title']
          del dic['src']
          del dic['abstract']
          
          with open(dataset_name+ '/' +dataset_name+ "_tokenizerBL1028.pred",'a+') as t:
              json.dump(dic,t)
              t.write('\n')
        for item in generated:
            print(item)
            tokenizer.add_tokens(item)
        f.close()
        t.close()

In [ ]:
filepath = ['kp20k_valid2k']#,'krapivin','duc',,'nus','inspec'

for i in filepath:
    get_pred_BART(i)

kp20k_valid2k ：0 / 2000
kp20k_valid2k ：50 / 2000
kp20k_valid2k ：100 / 2000
kp20k_valid2k ：150 / 2000
kp20k_valid2k ：200 / 2000
kp20k_valid2k ：250 / 2000
kp20k_valid2k ：300 / 2000
kp20k_valid2k ：350 / 2000
kp20k_valid2k ：400 / 2000
kp20k_valid2k ：450 / 2000
kp20k_valid2k ：500 / 2000
kp20k_valid2k ：550 / 2000
kp20k_valid2k ：600 / 2000
kp20k_valid2k ：650 / 2000
kp20k_valid2k ：700 / 2000
kp20k_valid2k ：750 / 2000
kp20k_valid2k ：800 / 2000
kp20k_valid2k ：850 / 2000
kp20k_valid2k ：900 / 2000
kp20k_valid2k ：950 / 2000
kp20k_valid2k ：1000 / 2000
kp20k_valid2k ：1050 / 2000
kp20k_valid2k ：1100 / 2000
kp20k_valid2k ：1150 / 2000
kp20k_valid2k ：1200 / 2000
kp20k_valid2k ：1250 / 2000
kp20k_valid2k ：1300 / 2000
kp20k_valid2k ：1350 / 2000
kp20k_valid2k ：1400 / 2000
kp20k_valid2k ：1450 / 2000
kp20k_valid2k ：1500 / 2000
kp20k_valid2k ：1550 / 2000
kp20k_valid2k ：1600 / 2000
kp20k_valid2k ：1650 / 2000
kp20k_valid2k ：1700 / 2000
kp20k_valid2k ：1750 / 2000
kp20k_valid2k ：1800 / 2000
kp20k_valid2k ：1850 / 20

In [ ]:
list1 = [["grid"], ["service", "discovery"], ["uddi"], ["web", "services"], 
["grid", "service", "discovery"], ["web", "services", "discovery"], ["web", "service", "discovery"], 
["scalable", "grid", "service", "discovery"], ["autonomous", "control"], ["dude", "architecture"], 
["uddi", "registries"], ["dude"], ["uddi", "registries", "thereby", "making"], ["uddi", "registries", "thereby"], 
["web", "services", "control"], ["uddi", "registries", "thereby", "making", "uddi"], 
["web", "services", "registries"], ["uddi", "registries", "thereby", "making", "distribution"], 
["uddi", "registries", "thereby", "making", "scaling"], ["uddi", "registries", "thereby", "making", "uddi", "registries"], 
["uddi", "registries", "thereby", "making", "uddi", "more"], ["<unk>"], 
["uddi", "registries", "thereby", "making", "uddi", "more", "scaling"], 
["uddi", "registries", "thereby", "making", "uddi", "registries", "thereby"],
["uddi", "registries", "thereby", "making", "uddi", "more", "robust"],
["uddi", "registries", "thereby", "making", "uddi", "registries", "thereby", "making"],
["uddi", "registries", "thereby", "making", "uddi", "registries", "thereby", "making", "uddi"],
["scalable", "web", "services"], ["scalable", "web"], ["web", "service"], ["scalable", "web", "service"], 
["scalable", "web", "services", "discovery"], ["scalable", "web", "service", "discovery"],
["uddi", "registries", "thereby", "making", "uddi", "registries", "thereby", "making", "uddi", "registries"],
["uddi", "registries", "thereby", "making", "uddi", "registries", "thereby", "making", "uddi", "more"],
["scalable", "web", "service", "discovery", "mechanisms"],
["uddi", "registries", "thereby", "making", "uddi", "registries", "thereby", "making", "uddi", "registries", "thereby"],
["uddi", "registries", "thereby", "making", "uddi", "registries", "thereby", "making", "uddi", "registries", "thereby", "making"],
["uddi", "registries", "thereby", "making", "uddi", "registries", "thereby", "making", "uddi", "registries", "thereby", "making", "uddi"],
["<unk>", "<unk>"], ["uddi", "registries", "thereby", "making", "uddi", "registries", "thereby", "making", "uddi", "registries", "thereby", "making", "uddi", "registries"], 
["uddi", "registries", "thereby", "making", "uddi", "registries", "thereby", "making", "uddi", "registries", "thereby", "making", "uddi", "more"],
["web"], ["scalable"], ["scalable", "web", "service", "discovery", "based"]]

GTKP = ["bamboo dht code", "uddi registry", "md", "distributed web service discovery architecture", 
"longest available prefix","autonomous control", "discovery", "scalability issue", 
"dht based uddi registry hierarchy", "grid computing", "grid service discovery", 
"deployment issue", "uddi", "web service", "soft state", "query", "dht", "case insensitive search", 
"qos based service discovery"]

"uddi","grid service discovery",["autonomous", "control"],"uddi", "registries",



["['grid computing', 'web service discovery', 'distributed hash tables', 'dht', 'uddi','scalability',
 'grid service discovery architecture', 'autonomous control','spatial data mining', 'peer to peer', 
 'parallel computing and distributed systems', 'internet of things']"]




print (len(list1))

45


In [10]:
ARTICLE_TO_SUMMARIZE = '''
Keyphrases are condensed summaries of text information, representing a document's theme and core idea. Keyphrase extraction approaches based on statistics and deep learning have received widespread attention. However, the extractive approaches have obvious limitations like keyphrases must occur in the document. However, a considerable amount of human-annotated keyphrases are absent keyphrases such that they are not occurring in the source document explicitly, which cannot be extracted by the extractive methods. Keyphrase generation is based on a sequence-to-sequence deep learning model, generating present and absent keyphrases from the semantic representation of the document. However, recall and precision of generated abstract keyphrases still need to be improved. We propose a two-step keyphrase generation model to solve these problems, which consists of a finetuned BART-based keyphrase generator, coupled with a BERT cross encoder which ranks generated keyphrases. Our model can generate absent keyphrases and shows superior results over the strong baselines in four widely used datasets.
'''
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], return_tensors='pt', padding=True, truncation=True,).to(device)


# Generate keyphrases
#model.config.min_length = 128
summary_ids = model.generate(inputs['input_ids'], max_length = 256, min_length = 64, num_beams = 11).to(device)  #length_penalty = 3.0, top_k = 5,, no_repeat_ngram_size=3, early_stopping=True,, min_length = 64,
bart_pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in summary_ids]
print (bart_pred)



['["keyphrase extraction", "keyphrase generation", "deep learning", "condensed summaries of text", "bodied conversational agents", "cross encoder", "finiteness", "recall", "precision", "present and absent keyphrases", "semantic representation", "strong baselines"]']


In [ ]:
key_Catseq = '''
		[5.3575]	[impedance coupling] 	 [2931, 1280] [correct!]
		[5.3575]	[content targeted advertising] 	 [368, 3326, 4019] [correct!]
		[6.7950]	[on line advertising] 	 [21, 387, 4019] 
		[6.9837]	[search based advertising] 	 [145, 27, 4019] 
		[8.6871]	[web page] 	 [149, 2019] 
		[22.1644]	[impedance coupling strategy] 	 [2931, 1280, 365] 
		[27.6107]	[content targeted advertising can] 	 [368, 3326, 4019, 32] 
		[29.4837]	[vocabulary impedance] 	 [3787, 2931] 
		[29.7378]	[computer science] 	 [176, 800] 
		[29.8833]	[impedance] 	 [2931] 
		[42.5659]	[content] 	 [368] 
		[42.7403]	[web] 	 [149] [correct!]
		[43.8703]	[text] 	 [564] 
		[44.0307]	[vocabulary] 	 [3787] 
		[44.0490]	[computer] 	 [176] 
    '''


Key_BART_Beam9 = ['algorithms', 'design', 'experimentation','measurement', 'information retrieval',
                  'content targeted advertising', 'impedance coupling', 'web search', 'tagging', 
                  'association','semantic', 'access', 'text', 'algorithm', 'informal', 'user','search']


key_BART_Beam7 = '''
['algorithms', 'experimentation','measurement', 'human factors', 
'content targeted advertising', 'impedance coupling', 'web page matching', 
'information retrieval', 'navigation', 'text mining','search', 'tagging', 
'algorithm', 'paper', 'informal', 'user']
'''

key_BART_Beam6 = '''
[\'algorithms\', \'experimentation\',\'measurement\', \'human factors\', 
\'content targeted advertising\', \'impedance coupling\', \'web search\', 
\'information retrieval\', \'text mining\',\'search based advertising\', 
"user\'s business", \'web page\', \'tagging\', \'contention\',\'strategies\', 
\'optimality\', \'association\',\'semantic\',\'matching\', 
\'data\', \'algorithm\', \'effect\', \'query\']
'''

Key_BART_Beam5 = '''
[\'content targeted advertising\', \'impedance coupling\', \'web search\', 
\'information retrieval\',\'search based advertising\', "user\'s business", 
\'web page matching\', \'text mining\',\'semantic web\', \'navigation\', 
\'association\', \'contention\',\'strategies\',\'method\', \'accuracy\', \'users\', 
\'optimality\',\'matching\',\'retrieval\', \'data\', \'algorithm\', \'query\']
'''

Key_BART_Beam3 = '''
[\'content targeted advertising\', \'impedance coupling\', \'web search\', 
\'information retrieval\',\'search based advertising\', "user\'s business", 
\'web page matching\', \'text mining\', \'data mining\', "aak\'s algorithm", 
\'information extraction\', \'query processing\', \'navigation\', \'algorithms\']
'''
Key_BART_No_Beam = '''
[\'content targeted advertising\', \'impedance coupling\', \'web search\'
]advertisement\']advertisementadvertisement\',\'search based advertising\', 
"user\'s business"]advertisementadvertisement\']... 
\'advertisement advertisement\', \'advertisement matching\', 
\'content based advertising\']. \'web page matching\', 
"adman\'s keyword", \'web site matching\',
'''


keywords = ["advertisements", "triggering page", "bayesian networks", "advertising", "matching", 
            "knn", "web", "content targeted advertising", "impedance coupling"]

B10_len200 = 
['[\'algorithms\', \'experimentation\',\'measurement\', \'performance\', \'theory\', 
\'content targeted advertising\', \'impedance coupling\', \'information retrieval\', \'web search\',
\'semantic web\', \'wordnet\',\'search engine\', \'user navigation\', \'on line advertising\', "user\'s business", 
\'web page\',\'vectorization\', \'association\', \'text\', \'algorithm\', \'effect\', \'evaluation\', \'query\', 
\'coupling\',\'scheme\', \'ad hoc\', \'combinational\', \'advertising\', \'pagerank\', \'all\',\'semi supervised learning\', 
\'classification\',\'matching\', \'clustering\', \'data mining\', \'dynamic programming\', \'decision trees\', \'learning\', 
\'internet\', \'interactive\',\'multimedia\', \'distributed\', \'demonstrate\', \'complexity\', \'utility\', \'profiles\', \
\'audience\', \'documentation\', \'information systems\', \'tools\',\'mobile computing\',\'resource\', \'business\',
\'\'information\',\'\'internet advertising\'] \'web\',\'social networks\', \'digital\'ads\', \'computer science\', \'']




BL1028 = '''
["precise", "yield", "contention", "vectorization", "method", "strategies", "accuracy", 
"semantic", "association", "web page", "matching", "text", "operability", "algorithm", "access", 
"effect", "informal", "evaluation", "user", "search", "couples", "targeted advertising", "ad hoc networks", 
"information retrieval", "on line advertisement", "contextual advertising"]advertisement", "advertisement", 
"keyword matching", "advertisement correlation", "advertisement conflict resolution", "content targeted advertising", "<digit> <digit>"]']

'''
['["precise", "yield", "contention", "vectorization", "method", "accuracy", "semantic", "association"
, "web page", "matching", "text", "operability", "algorithm", "access", "effect", "informal", 
 "evaluation", "user", "search", "couples", "business"]']



BL_512_128 = '''
['[\'algorithms\', \'experimentation\',\'measurement\', \'performance\', \'content targeted advertising\', 
\'impedance coupling\', \'on line advertising\', "user\'s navigation",\'semantic web\', \'web page\', 
\'text\', \'attributes\', \'account\', \'algorithm\', \'effect\', \'evaluation\',\'search\', \'coupling\',
\'vectorization\', \'association\',\'matching\',\'scheme\', \'business\', \'information retrieval\', \'query\',
 \'ad hoc\', \'combinational\', \'user\', \'cluster\', \'advertising\', \'pagerank']
'''






B20_len256 = '''
["['contention','method', 'accuracy','semantic', 'web page', 'computation','matching', 
'text', 'account', 'access', 'algorithm', 'informal', 'effect', 'evaluation', 'user','search', 
'couples', 'association', 'online advertising', 'advertising','scheme', 'information retrieval', 
'query', 'content targeted', 'pagerank','vectorization', 'business', 'ad hoc', 'ranking', 'integrability', 
'classifiation', 'robotics', 'circ','semi supervised learning', 'natural language processing', 
'world wide web','syntactic similarity', 'clustering', 'internet', 'tools', 'interactive systems', 
'computer science', 'data', 'profiles', 'utility', 'complexity','systems', 'transparency', 
'demonstrate',''web','resource', 'distributed networks','software','ads',''utimedia', 'privacy','mobile', 
'un''utilities', 'intelligent interfaces','advertisements','social advertisement', 'virtual '"]
'''


B30_len512 = '''
['[\'contention\',\'method\', \'accuracy\',\'semantic\', \'web page\', \'association\', \'computation\',
\'matching\', \'text\', \'account\', \'algorithm\', \'informal\', \'evaluation\', \'user\',\'search\', \'couples\',
\'vectorization\', \'business\', \'advertising\', \'effect\', \'information retrieval\',\'scheme\', 
\'content targeted advertising\', \'world wide web\', \'query\', \'pagerank\', \'embedding\', 
\'integrability\', \'ad hoc\', \'robotics\', \'programming and algorithm theory\',\'\'query optimization\', 
\'classification\', \'clustering\', \'feature extraction\', \'data mining\', \'ranking\', \'tools\', \'profiles\',
 \'demonstrate\', \'internet\',\'segmentation techniques\',\'similarity\', \'utility\', \'interactive systems\', 
 \'all\', \'documentation tools\',\'social networks\',\'resource\', \'computer science\', \'electronic commerce\', 
 \'distributed applications\', \'contin advertisements\',\'multimedia\',\'multi user interfaces\',\'systems\',\'\'user interface\', 
 \'privacy\',\'mobile computing\', \'audience\', \'transactions\',\'software\',\'ads\', \'un\'\'c " advertisement\',
 \'advertisements\', " ad enterprises\', \'trust ads\'ad interfaces\', " ads ad\'advertisements\'] advertisements\'ads\',
 \'\' advertising\'advertisement\'interfaces\'] \'\', advertising\'] interfaces and advertising advertisements\', advertisements\',\', 
 \'\',\'interfaces\', advertising interfaces\', ad ads\', ads\', advertisement\', advertisement ads\', advertisers\', 
 \'imedia advertisements\', advertisement advertisements\', ad advertisements\', advertising advertisements\'] 
 \'internet advertising\'advertisements\']\',\'keywords\', \'environment\',\'advertisement\'\'internet advertisements\'\' 
 \'web advertising\'\' advertising ads\', ad\'\' advertisers\', advertisement\'ads\']\'advertisements \'internet ads\', 
 "\', \'", \'internet\'interfaces\'\'document\'\' advertisements\'advertisement ad\'ads\'\'advertising\'\' ad\',
 \'advertising\'advertisers\'\' advertisement\', " \'internet advertisement\'advertising ad\'ad\'advertisement advertisements\
 'ads ads\'ads ad\'advertisements ad\'advertisers\']\'\'business\'\' ads and \'internet ad\'advertising advertising\'ads advertisements\',
  ads\'advertisement advertising\'ad ads\'ad advertisements\'advertisements and\'\' commercials\',\'ad']
'''

B50_len512 = '''['[\'contention\',\'strategies\', \'accuracy\',\'semantic\', \'web page\', \'computation\',
\'matching\', \'addressing\', \'account\', \'access\', \'algorithm\', \'informal\', \'evaluation\', \'user\',
\'search\', \'couples\',\'vectorization\', \'association\', \'text\', \'advertising\', \'effect\',\'scheme\', 
\'world wide web\', \'information retrieval\', \'content targeted advertising\', \'pagerank\', \'classifiation\', 
\'ad hoc\', \'ranking\', \'query\', \'integrability\', \'all\',\'semi supervised learning\', 
\'interactive learning environments\', \'clustering\', \'unsupervised learning algorithms\', \'learning\', 
\'data mining\', \'computer science\', \'profiles\', \'collaborative filtering\', \'tools\', \'electronic commerce\',
\'multimedia\', \'internet\', \'privacy\', \'utility\', \'embedded systems\', \'business\',\'\'web\', \'contextual information\',
\'\'user interfaces\',\'advertisements\',\'social networks\',\'systems\', \'demonstrate\', \'contin\'\'web ads\', \'transactions\',
\'keywords\',\'multi channels\', \'dist\'\' advertising advertisement\', \'customisation\', \'global ads " advertisements\', 
" ads\'ad advertisements\'] advertising\', advertisements\'ads\'] interfaces advertising\'advertisement\'interfaces\']\']
 ad\'commercials\'interfaces\', ad ad advertisements\', advertisement advertisements\', ads interfaces\'advertisers\', \'& ad interfaces\', 
 " ad advertising ads\', ads\', advertising advertisements\', advertisements\', \'\', \'imedia advertisements\',\',\'commercials\', 
 \'commercial ad\'advertisements ad\'\'internet advertisements\', advertising\'\' advertising advertising advertisements\'] 
 \'internet advertising \'internet ads\', advertisements\'\'\'ads\', advertisement\'\' advertisers\', advertisement ads\', ad\',
 \'enterprises\', \'document\',\'advertisement\'advertisements\'ads\'\'advertising ads\', "\'\' advertisement\', ad \'internet\'
 \' advertising\'advertisers\'\' ad\'ads\']\'\' advertisements and\'\'business\'\'", \'internet advertisement \'internet ad\
 'advertisement advertisements\'advertisement ads\'ads ad\'advertisements ad ads\'advertisement ad\'ad ads\', advertising ad\'advertising\',
  ads\'advertising ads\'ad advertisements\'advertising advertisements\'ad advertising\'advertisement advertising \'']

'''

B1_len512 = '''
['[\'web advertising\', \'content targeted advertising\', "user\'s information"] \'impedance coupling\'] \'web page matching\'] 
\'wordnet\'] \'information retrieval\'] \'query optimization\', \'information extraction\'] \'text mining\', 
\'text classification\', \'web search\', \'query processing\', \'user interface\'] \'user navigation\',
 \'on line advertising\', \',\'vectorization\',\'strategies\', \'accuracy\',\'semantic\',\'matching\', \'algorithm\', 
 \'effect\', \'couples\', \'advertising\',\'search\', \'internet\'] \'effectiveness\'] \'algorithms\', \'world\'\'webpage\', 
 \'ad hoc\'] \'internet\',\'\'\', "\'\' \'web\'"#\'advertisement \'"]\'" "………………………………>>>>>>>>\'ad \'……………………\'advertisement\'\'
  \'all\'"# ad\'ads\', \'ape\'advertisements\',\'\'ape ad "\'advertisers\', advertisement\', \'all\'ubiquitous 
  \'ube\'commercials\', ad hoc \'ould \'allould advertisements\'multimedia \'internet\'clutter \
  'document\'advertising\'interfaces\', \'document \'& \'az\'autop\'advertisers\', \'edge\'ads\',\'advertisement\', 
  advertisements\']\'advertisements\', ads \'", \'& ads \'al \'edge\'emergency\'ad\'\'document ad \'document\', 
  advertisement ad\'advertisement ad ad ad\'advertisers\'internet\'enterprise \'document\'\' ads\'\'internet ad\'ads ad \'",
  \'\' advertisement ads \'\', \'", \'", "\', ad \'\', "\'\'", ad \'[\'\'\' advertisers\']\'\' advertisements\'\' ad ad ad ads\'
  ads", \'internet advertisement\'ads\']\'&\'" ad\',\'ads ads\'advertisement advertisement \'internet ads\'ad ads\', 
  "\',\'advertisements ad\'ad\', "",\'ads advertisement\'advertisement advertisements\'ads advertisements\'advertisement advertising\'
  \'advertising\'\' advertising\'ads advertising \'", ads \'",\',\'ad", \'", advertisement\'ad advertisement \'",", \'document ads 
  \'internet advertising\'ad advertising\'advertisement",\'advertisement "\'ads " \'internet advertisements\'ad advertisements \'", 
  advertisements \'internet", \'c\'\' "\'ad']
'''

B9_len512 = '''
['[\'algorithms\', \'experimentation\',\'measurement\', \'performance\', \'theory\', \'content targeted advertising\', 
\'impedance coupling\', \'information retrieval\',\'semantic web\', \'web search\', \'on line advertising\', 
"user\'s business", \'web page\', \'wordnet\',\'search engine\', \'user navigation\', \'query\',\'vectorization\',
\'strategies\', \'accuracy\', \'optimality\', \'text\', \'algorithm\', \'effect\', \'coupling\',\'scheme\', \'advertising\', 
\'ranking\', \'combinational\', \'ad hoc\']\'search\', \'pagerank\', \'classification\', \'cluster\', \'data mining\', 
\'tools\', \'internet\',\'searchers\',\'\'query languages\',\'similarity\',\'resource\',\'ads\', \'demonstrate\', \'utility\', 
\'audit\', \'profiles\', \'all\', \'online\', \'business\', \'dynamic programming\', \'documentations\', \'database\', 
\'computer science\', \'interactive\', \'natural \'information systems\', \'contin information\', \'distributed systems\',
\'query\'ads\', advertisements\',\'advertisements\', " ad advertisements ads\'ad\'advertisements\'] " advertisements\'advertisement\',
\'interfaces\', ads\'] advertising\'] ads ads advertising\'\'\'ads ad ad advertisement \'\'] advertisements ad ad advertisements\', 
advertisements advertisement advertisements\', advertisers\', advertisements advertisements\', ad \'\', \'\',\', ad ads\', " 
interfaces\'advertising advertisements\', advertisement ads\', advertisements\',\',\'strate\'advertisers\', \'imedia advertisements 
\'internet \'\', advertisement\', ads\', advertisement\'advertisements ad\'enterprises\', \'circ\'advertisement\'\' advertisements ads\',
\'advertisements\'\'\'advertisers\', advertisement advertisements\'advertisement\', advertisement ad\'\' advertisement advertisements\'] 
ad\',\'advertisement ads \'document\', \'",\'\'internet advertisements\', ads ads\', ad ads\'\' ads \'\', "\', \'
[\'internet advertising\'advertisements\'] \'internet advertisement\'ads\']\'\' ad\'advertisers\'\' advertising\'\'advertising\'\' 
"\'\'document\'\'\', ads\'ads ads\'advertisement advertising\'ads advertisements\', advertising ads\',\', ad \'internet ads\', 
advertising \'internet advertising advertisements\'ads ad\'ads advertising\'advertisement advertisement\'advertising\',']
'''

large-bart = ['algorithms', 'experimentation','measurement', 'performance', 'content targeted advertising', 
              'impedance coupling','semantic web', 'information retrieval', 'web search', 'text mining', 
              'wordnet','search engine', 'user interface', 'on line advertising']

Key_BART_Beam9 = ['algorithms', 'design', 'experimentation','measurement', 'information retrieval',
                  'content targeted advertising', 'impedance coupling', 'web search', 'tagging', 
                  'association','semantic', 'access', 'text', 'algorithm', 'informal', 'user','search']

B9_len256 = '''
['</s>[\'algorithms\', \'experimentation\',\'measurement\', \'performance\', \'theory\', 
\'content targeted advertising\', \'impedance coupling\', \'information retrieval\',\'semantic web\', 
\'web search\', \'on line advertising\', "user\'s business", \'web page\', \'wordnet\',\'search engine\', 
\'user navigation\', \'query\',\'vectorization\',\'strategies\', \'accuracy\', \'optimality\', 
\'text\', \'algorithm\', \'effect\', \'coupling\',\'scheme\', \'advertising\', \'ranking\', 
\'combinational\', \'ad hoc\']\'search\', \'pagerank\', \'classification\', \'cluster\', \'data mining\', 
\'tools\', \'internet\',\'searchers\',\'\'query languages\',\'similarity\',\'resource\',\'ads\', \'demonstrate\', 
\'utility\', \'audit\', \'profiles\', \'all<s>\', \'online\', \'business\', \'dynamic programming\',
 \'documentations\', \'database\', \'computer science\', \'interactive\', \'natural<s> \'information systems\',
  \'contin<s><s><s> information\', \'distributed systems\']<s> \'query<s><s> \'<s><s> ads\',<s><s> advertisements\', \'<s></s>']
